<a href="https://colab.research.google.com/github/mahima124/githubtest/blob/main/PIC_modified_hot_electrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

def getAcc(pos, Nx, boxsize, n0, Gmtx, Lmtx):
    Nh_hot = pos.shape[0]
    #Nh_hot=pos.shape[0]
    dx = boxsize / Nx
    j = np.floor(pos / dx).astype(int)
    jp1 = j + 1
    weight_j = (jp1 * dx - pos) / dx
    weight_jp1 = (pos - j * dx) / dx
    jp1 = np.mod(jp1, Nx)
    n = np.bincount(j[:, 0], weights=weight_j[:, 0], minlength=Nx)
    n += np.bincount(jp1[:, 0], weights=weight_jp1[:, 0], minlength=Nx)
    n *= n0 * boxsize / N / dx

    phi_grid = spsolve(Lmtx, n - n0, permc_spec="MMD_AT_PLUS_A")
    E_grid = -Gmtx @ phi_grid
    E = weight_j * E_grid[j] + weight_jp1 * E_grid[jp1]
    #print("electric field",E)
    a = -E

    return a

def main():
    N = 40000
    Nx = 400
    t = 0
    tEnd = 500
    dt = 1
    boxsize = 50
    n0 = 1
    vb = 3.5
    vth = 1
    A = 0.1
    hot_electron_percentage = 20

    plotRealTime = True

    np.random.seed(42)
    pos = np.random.rand(N, 1) * boxsize
    Nh_hot = int(N * hot_electron_percentage / 100)+int(N/2)
    Nh=int(N/2)
    #N1=int(N/2)

    # Add perturbation for thermal electrons
    thermal_velocity = vth * np.random.randn(Nh_hot, 1)
    vb1=vb
    vel =  thermal_velocity+vb1
    #vel1=vb1
    vel[Nh:]=0
    vel[Nh:] += A * np.sin(2 * np.pi * pos[Nh:] / boxsize)
    #vel1[N1:]=0
    #vel1[N1:] += A * np.sin(2 * np.pi * pos[N1:] / boxsize)



    dx = boxsize / Nx
    e = np.ones(Nx)
    diags = np.array([-1, 1])
    vals = np.vstack((-e, e))
    Gmtx = sp.spdiags(vals, diags, Nx, Nx)
    Gmtx = sp.lil_matrix(Gmtx)
    Gmtx[0, Nx - 1] = -1
    Gmtx[Nx - 1, 0] = 1
    Gmtx /= (2 * dx)
    Gmtx = sp.csr_matrix(Gmtx)

    diags = np.array([-1, 0, 1])
    vals = np.vstack((e, -2 * e, e))
    Lmtx = sp.spdiags(vals, diags, Nx, Nx)
    Lmtx = sp.lil_matrix(Lmtx)
    Lmtx[0, Nx - 1] = 1
    Lmtx[Nx - 1, 0] = 1
    Lmtx /= dx**2
    Lmtx = sp.csr_matrix(Lmtx)

    acc = getAcc(pos, Nx, boxsize, n0, Gmtx, Lmtx)
    Nt = int(np.ceil(tEnd / dt))

    fig = plt.figure(figsize=(6, 6), dpi=120)

    for i in range(Nt):
        vel += acc * dt / 2.0
        pos += vel * dt
        pos = np.mod(pos, boxsize)
        acc = getAcc(pos, Nx, boxsize, n0, Gmtx, Lmtx)
        vel += acc * dt / 2.0
        t += dt

        if plotRealTime or (i == Nt - 1):
            plt.cla()
            plt.scatter(pos[:Nh], vel[:Nh], s=0.4, color='red', alpha=0.5)
            #plt.scatter(pos[:N1], vel[:N1], s=0.4, color='yellow', alpha=0.5)
            plt.scatter(pos[Nh:], vel[Nh:], s=0.4, color='green', alpha=0.5)

            #plt.scatter(pos[N1:], vel1[N1:], s=0.4, color='blue', alpha=0.5)

            plt.axis([0, boxsize, -6, 6])

            plt.pause(0.001)

    plt.xticks(fontweight='bold', fontsize=20)
    plt.yticks(fontweight='bold', fontsize=20)
    plt.xlabel('x', fontweight='bold', fontsize=25)
    plt.ylabel('v', fontweight='bold', fontsize=25)
    plt.savefig('pic.png', dpi=240)
    plt.show()

    return 0

if __name__ == "__main__":
    main()

ValueError: operands could not be broadcast together with shapes (8000,1) (20000,1) (8000,1) 